In [1]:
from datasets import load_dataset
import os 
PATH = os.path.join("..","data") 
ds = load_dataset("dair-ai/emotion", "split",cache_dir = PATH )

README.md: 0.00B [00:00, ?B/s]

f:\python_file_install\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\datasets--dair-ai--emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
import pandas as pd 
base_path = "../data/train_data"
os.makedirs(base_path , exist_ok = True)
PATH_TRAIN = os.path.join(base_path,"train.csv")
PATH_VAL = os.path.join(base_path,"val.csv")
PATH_TEST = os.path.join(base_path,"test.csv")
train = ds["train"][:400]
val = ds["validation"][:200]
test = ds["test"][:100]
# pd.DataFrame(train).to_csv(PATH_TRAIN)
# pd.DataFrame(val).to_csv(PATH_VAL)
# pd.DataFrame(test).to_csv(PATH_TEST)

In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts.prompt import PromptTemplate
extender_template = PromptTemplate.from_template(
    """
    You need to extend the sentence to 1000 words,like emotion journaling without changing sentiments and return only the sentence
    Here is sentence and its sentiment
    sentence : {sentence}
    sentiment : {sentiment}

    Output:
    1) 1000 words extended sentence
    """
)

In [5]:
# Connect LLM
from dotenv import load_dotenv
load_dotenv()
os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
chain = extender_template | llm

In [6]:
idx_2_class = {index : value for index , value in enumerate(ds["train"].features["label"].names)}
idx_2_class

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [32]:
train_data = {
    "text": [],
    "label": []
}

In [ ]:
import time
for text , label in zip(train["text"][268:] , train["label"][268:]):
    label = idx_2_class[label]
    inputs = {"sentence": text, "sentiment": label}
    response = chain.invoke(inputs)
    train_data["text"].append(response.content)
    train_data["label"].append(label)
    time.sleep(10)  

In [31]:
response.content

"I didn't feel humiliated, which was a surprise to me, considering the circumstances that had just unfolded, and as I reflected on it, I realized that it was a testament to my growth and resilience, and it made me think about all the times I had felt humiliated in the past, and how it had affected me, and I had to admit that it was a pretty terrible feeling, and one that I didn't wish on anyone, but at the same time, I was grateful that I had been able to navigate this situation without feeling that way, and it made me wonder if I had finally reached a point where I was comfortable with myself, and wasn't as concerned with what others thought of me, and that was a pretty liberating feeling, and it made me think about the concept of humiliation, and how it can be used as a tool to control and manipulate people, and how it can be a powerful motivator for some, but for me, it was just a reminder that I had come a long way, and that I still had a lot to learn, and I was okay with that, and

In [41]:
len(train_data["text"])

400

In [44]:
base_path_ext = "../data/train_data_ext"
os.makedirs(base_path_ext, exist_ok=True)
PATH_TRAIN_EXT = os.path.join(base_path_ext,"train.csv")
pd.DataFrame(train_data).to_csv(PATH_TRAIN_EXT, index=False)

In [47]:
test_data = {"text": [], "label": []}

In [48]:
for text, label in zip(val["text"], val["label"]):
    label = idx_2_class[label]
    inputs = {"sentence": text, "sentiment": label}
    response = chain.invoke(inputs)
    test_data["text"].append(response.content)
    test_data["label"].append(label)
    time.sleep(10)

In [49]:
PATH_TRAIN_EXT = os.path.join(base_path_ext, "val.csv")
pd.DataFrame(test_data).to_csv(PATH_TRAIN_EXT, index=False)

In [50]:
val_data = {"text": [], "label": []}

In [51]:
for text, label in zip(test["text"], test["label"]):
    label = idx_2_class[label]
    inputs = {"sentence": text, "sentiment": label}
    response = chain.invoke(inputs)
    val_data["text"].append(response.content)
    val_data["label"].append(label)
    time.sleep(10)

In [52]:
PATH_TRAIN_EXT = os.path.join(base_path_ext, "test.csv")
pd.DataFrame(val_data).to_csv(PATH_TRAIN_EXT, index=False)